In [88]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

In [92]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/heatwaves/Adilabad.csv")

In [93]:
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'date' column as the index
df.set_index('Date', inplace=True)
df.head()

,PM10,NOx,SO2
Date,,,
2016-01-01,62.0,14.0,3.0
2016-01-02,68.0,14.0,3.0
2016-01-03,71.0,14.0,3.0
2016-01-04,70.0,14.5,3.5
2016-01-05,69.0,15.0,3.5


In [94]:
df1=df['PM10']
df1=df1.to_frame()
df2=df['NOx']
df2=df2.to_frame()
df3=df['SO2']
df3=df3.to_frame()

In [95]:
model1 = ARIMA(df1, order=(1,1,1))
model1_fit = model1.fit()
model2 = ARIMA(df2, order=(1,1,1))
model2_fit = model2.fit()
model3 = ARIMA(df3, order=(1,1,1))
model3_fit = model3.fit()

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr

In [96]:
def month_diff(dt1,dt2):
  from datetime import datetime
  from dateutil.relativedelta import relativedelta
  date1_str = dt1
  date2_str = dt2
  date1 = datetime.strptime(dt1, '%Y-%d-%m').date()
  date2 = datetime.strptime(dt2, '%Y-%d-%m').date()
  delta = relativedelta(date2, date1)
  months_diff = delta.months + 12 * delta.years
  return months_diff

In [97]:
s=input("enter starting date of month you want to predict in yyyy-dd-mm format: ")
diff=month_diff('2022-01-12',s)
PM10prediction = model1_fit.forecast(diff)
NOxprediction = model2_fit.forecast(diff)
SO2prediction = model3_fit.forecast(diff)


enter starting date of month you want to predict in yyyy-dd-mm format: 2023-01-04


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [98]:
PM10prediction = PM10prediction.to_numpy()
NOxprediction = NOxprediction.to_numpy()
SO2prediction = SO2prediction.to_numpy()

In [99]:
PM10=PM10prediction[diff-1]
NOx=NOxprediction[diff-1]
SO2=SO2prediction[diff-1]

In [100]:
print(PM10,NOx,SO2)

61.38528367648924 25.769091118720212 9.116513307068093


In [81]:
def get_PM10_subindex(x):
    if x <= 50:
        return x
    elif x <= 100:
        return x
    elif x <= 250:
        return 100 + (x - 100) * 100 / 150
    elif x <= 350:
        return 200 + (x - 250)
    elif x <= 430:
        return 300 + (x - 350) * 100 / 80
    elif x > 430:
        return 400 + (x - 430) * 100 / 80
    else:
        return 0

In [82]:
def get_NOx_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 180:
        return 100 + (x - 80) * 100 / 100
    elif x <= 280:
        return 200 + (x - 180) * 100 / 100
    elif x <= 400:
        return 300 + (x - 280) * 100 / 120
    elif x > 400:
        return 400 + (x - 400) * 100 / 120
    else:
        return 0

In [83]:
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0

In [101]:
PM10=get_PM10_subindex(PM10)
NOx=get_NOx_subindex(NOx)
SO2=get_SO2_subindex(SO2)

In [102]:
AQI=max(PM10,NOx,SO2)
AQI

61.38528367648924

In [103]:
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    else:
        return "Severe"

In [104]:
AQI_STATUS=get_AQI_bucket(AQI)
AQI_STATUS

'Satisfactory'